In [ ]:
#!pip install qiskit_ibm_provider
#!pip install seaborn
#!pip install pylatexenc

In [ ]:
from qiskit_ibm_provider import IBMProvider

# Providers

In [ ]:
# do it only once
IBMProvider.save_account(token='MY_TOKEN', overwrite=True)

In [ ]:
provider = IBMProvider()

In [ ]:
print(provider.backends())

In [ ]:
for backend in provider.backends():
    try:
        print(backend.name, backend.num_qubits, backend.status().status_msg, backend.status().pending_jobs)
        print(backend.coupling_map)
        instructions = set()
        for inst, _ in backend.instructions:
            if isinstance(inst.name,str) and inst.name not in backend.configuration().basis_gates:
                instructions.add(inst.name)
        print("\tsupports: ", backend.configuration().basis_gates, "+", instructions)
    except:
        pass

In [ ]:
simulator_backend = provider.get_backend('ibmq_qasm_simulator')
kyoto_backend = provider.get_backend('ibm_kyoto')

In [ ]:
from qiskit.tools.visualization import plot_gate_map
print(kyoto_backend.configuration().coupling_map)
plot_gate_map(kyoto_backend)

In [ ]:
#Can we check coupling map for simulator?
###ENTER CODE HERE
###END CODE

# Simulation

In [ ]:
# Our first circuit !
from qiskit import QuantumCircuit
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()

circuit.draw(initial_state=True)
circuit.draw(output='mpl', style="iqp") # style="clifford"

In [ ]:
from qiskit.quantum_info import Statevector
from qiskit import Aer
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
local_simulator = Aer.get_backend('aer_simulator')
local_kyoto_simulator = AerSimulator.from_backend(kyoto_backend)
job = local_simulator.run(circuit, shots=1024)

In [ ]:
print(job.status())

In [ ]:
result_qasm = job.result()
print(result_qasm.get_counts())

In [ ]:
result_kyoto = local_kyoto_simulator.run(circuit, shots=1024).result()
print(result_kyoto.get_counts())

In [ ]:
plot_histogram([result_qasm.get_counts(), result_kyoto.get_counts()], legend=["ideal", "noisy"])

In [ ]:
## Calculate fidelity with precision linked to sampling noise

###ENTER CODE HERE
###END CODE

In [ ]:
# let us check statevector now
circuit2 = QuantumCircuit(2)
circuit2.h(0)
circuit2.cx(0, 1)
circuit2.save_statevector()

In [ ]:
from qiskit.tools.visualization import plot_state_qsphere
sv1 = local_simulator.run(circuit2).result().get_statevector()
plot_state_qsphere(sv1)

In [ ]:
sv1.draw('latex', prefix='The\ state\ vector:')

In [ ]:
## Can we do the same on noisy simulator? comment...

###ENTER CODE HERE
###END CODE

# Conditional X gate

Using `c_if` - you can make a gate conditional in your circuit - syntax is:

<code>
circuit.x(qreg).c_if(creg, 1)
</code>

Modify the circuit above to add a conditional X gate on the second qubit, if the value of the register is 0, what is the new circuit?

In [ ]:
qc_ghz = QuantumCircuit(3)
###ENTER CODE HERE
###END CODE

# Transpilation, Compilation, Assembling

You can use the following to check the size of your circuit:
<code>
qc.width()
qc.count_ops()
qc.size()
qc.depth()
</code>

Build a GHZ state $|000\rangle+|111\rangle$ and check these different values...

In [ ]:
###ENTER CODE HERE
###END CODE

print(f"initial: width={qc_ghz.width()}, counts_ops={qc_ghz.count_ops()}, size={qc_ghz.size()}, depth={qc_ghz.depth()}")

In [ ]:
# do the same on the decomposed circuit
qc_basis = qc_ghz.decompose()
print(f"decompose: width={qc_basis.width()}, counts_ops={qc_basis.count_ops()}, size={qc_basis.size()}, depth={qc_basis.depth()}")
qc_basis.draw(output='mpl')

In [ ]:
# ok - let transpile now the circuit and checks what it becomes on physical hardware
from qiskit.compiler import transpile
transp_3 = transpile(qc_ghz, kyoto_backend, optimization_level=3)
transp_3.draw(output='mpl')
print(f"transpiled: width={qc_basis.width()}, counts_ops={qc_basis.count_ops()}, size={qc_basis.size()}, depth={qc_basis.depth()}")

In [ ]:
# is there difference with optimization level 1 and 2
###ENTER CODE HERE
###END CODE

In [ ]:
# How is the circuit transpiled on the actual layout?
from qiskit.visualization import plot_circuit_layout
plot_circuit_layout(transp_1, kyoto_backend)

In [ ]:
# Let us check on Assembling of transp_1

qobj_1 = assemble(transp_1)
print(qobj_1)

In [ ]:
# How does conditional gates assemble

###ENTER CODE HERE
###END CODE

# Remote jobs...
To retrieve a job from IBMQ you can just use `job = provider.runtime.job('JOB_ID')`... `job` is the actual job object you could run locally.

In [2]:
# take a IBMQ provider (to be quick a simulator) and launch a bell-state preparation circuit directly on the cloud
# retrieve the result locally using job-id and display the result...